
\begin{align*}
u_{tt} &= u_{xx} \\
u_{tt}(t,x) &= \frac{u(t,x+ \Delta x)-2u(t,x)+u(t,x- \Delta x)}{\Delta x^{2}} \\
u_{tt}(t,x) + \frac{2}{\Delta x^{2}}u(t,x) &= \frac{u(t,x+ \Delta x)+u(t,x- \Delta x)}{\Delta x^{2}} \\
\end{align*}
Introduce: $q = \frac{\sqrt{2}}{\Delta x}$.
\begin{align*}
u_{tt}(t,x) + 2 q u_t(t,x) + q^{2}u(t,x) &= q^{2}\frac{u(t,x+ \Delta x)+u(t,x- \Delta x)}{2}+ 2 q u_t(t,x) \\
\end{align*}
Green's functions:

\begin{align*}
u(t,x) &= e^{-qt}(qt+1) u(0,x) + t e^{-qt}u_t(0,x)
+  \int_{0}^{t} (t-s) e^{q(s-t)} 
\left[\frac{q^{2}}{2} (u(s,x+\Delta x)+u(s,x-\Delta x)) + 2 q u_t(s,x)\right]  ds
\end{align*}

Now take time derivative: 
\begin{align*}
u_t(t,x) &= -q^{2} t e^{-qt} u(0,x) + (1-qt) e^{-qt}u_t(0,x) +  \int_{0}^{t} (1-qt+qs) e^{q(s-t)} \left[\frac{q^{2}}{2} (u(s,x+\Delta x)+u(s,x-\Delta x)) + 2 q u_t(s,x)\right]  ds
\end{align*}

Still need to try out removing $u_t$ with integration by parts and splitting the integral to monotone terms for importance sampling.

This is assuming you know noisy $u_t$

wolframalpha: y''(t) +2ay'(t)+a^2 y(t) = delta(t-s) , y(0) =0 , y'(0)=0, t>0 ,s>0 <br>
$$
y''(t) +2ay'(t)+a^2 y(t) = \delta(t-s) , y(0) =0 , y'(0)=0, t>0 ,s>0
.
$$ 
solution : 
$$
y(t) = (s - t) e^{a (s - t)} (θ(-s) - θ(t - s))
.
$$ 

wolframalpha: {y''(t) + 2 a y'(t) + a^2 y(t) = 0, y(0) = 1, y'(0) = 0, t>0} <br>
$$
y''(t) + 2 a y'(t) + a^2 y(t) = 0, y(0) = 1, y'(0) = 0, t>0
.
$$ 
solution: 
$$
y(t) = e^{-a t} (a t + 1).
$$ 

wolframalpha: {y''(t) + 2 a y'(t) + a^2 y(t) = 0, y(0) = 0, y'(0) = 1, t>0}
$$
y''(t) + 2 a y'(t) + a^2 y(t) = 0, y(0) = 0, y'(0) = 1, t>0
.
$$ 
solution:
$$
y(t) = t e^{-a t}
.
$$ 

wolframalpha: d/dt( integral_0^t (t - s) exp(a (s - t)) f(s) ds) = integral_0^t (e^(a (s - t)) f(s) - a e^(a (s - t)) (t - s) f(s)) ds <br>
solution:
\begin{align*}
 \frac{\partial}{\partial t}\left(\int_0^t(t-s) \exp (a(s-t)) f(s) d s\right)=\int_0^t\left(e^{a(s-t)} f(s)-a e^{a(s-t)}(t-s) f(s)\right) d s   
\end{align*}




In [1]:
from random import random as U
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

# def u(t,x): return x**2 + 2*t #needs to satisfy wave equation
# def ut(t,x): return  2 #needs to satisfy wave equation

# def u(t,x): return x*t**2 + x**3/3 #needs to satisfy wave equation
# def ut(t,x): return 2*x*t  #needs to satisfy wave equation

def u(t,x): return np.sin(x+t) #needs to satisfy wave equation
def ut(t,x): return np.cos(x+t)  #needs to satisfy wave equation

def P1(t,q): return (q*t+1)*np.exp(-q*t) 
def P2(t,q): return t*np.exp(-q*t) 
def GG(t,s,q): return (t-s)*np.exp(q*(s-t)) 
count = 0

def Z0(t,x,dx,dt):
    q =np.sqrt(2)/dx
    s = t*U()
    xnew = x+dx if U()<1/2 else x-dx
    sol = P1(t,q)*u(0,x)
    sol += P2(t,q)*ut(0,x)
    sol += t*GG(t,s,q) * 2 *q * ut(s,x)
    if t/dt>1:
        sol+=t*GG(t,s,q)*q**2*Z0(s,xnew,dx,dt)
    elif U()<t/dt:
        sol+=dt*GG(t,s,q)*q**2*Z0(s,xnew,dx,dt)
    return sol

def Z(t,x,dx,dt):
    if t< dt: return Z0(t,x,dx,dt) 
    t0 = t-dt
    q =np.sqrt(2)/dx
    s = dt*U()+t0
    xnew = x+dx if U()<1/2 else x-dx
    samp = 5/10
    if U()<samp: 
        sol = P1(dt,q)*Z(t0,x,dx,dt)/samp
    else:
        sol=dt*GG(t,s,q)*q**2*Z(s,xnew,dx,dt)/(1-samp)
    
    rr_rate = q if 1/q <1 else 1

    if 1/rr_rate>U():
        rr = rr_rate
        global count
        count +=1
        sol += rr*dt*P2(dt,q)*ut(t0,x) 
        sol += rr*dt*GG(t,s,q)* 2 *q *ut(s,x)

    return sol

def update_histogram(t,x,dt, dx,nsim):
    nsim = 10**int(nsim)
    global count
    count = 0
    zz = np.array([Z(t, x, dx,dt) for _ in range(nsim)])
    avg_label.value = f"Updated Average of zz: {np.average(zz):.2f}, {u(t,x):.2f}"
    counter.value = f"Updated Average of count: {count/nsim} "
    plt.hist(zz, bins=50)
    plt.xlabel("Values of Z")
    plt.ylabel("Frequency")
    plt.title("Histogram of Z")
    plt.show()

dt_slider = widgets.FloatSlider(value=1, min=0.1, max=10, step=0.01, description='dt:')
dx_slider = widgets.FloatSlider(value=1, min=0.1, max=10, step=0.01, description='dx:')
t_slider = widgets.FloatSlider(value=0, min=0, max=10.0, step=0.1, description='t:')
x_slider = widgets.FloatSlider(value=0, min=-50, max=50, step=1, description='x:')
nsim_slider = widgets.FloatSlider(value=0, min=0, max=5, step=1, description='nsim:')

avg_label = widgets.Label(value="")
counter = widgets.Label(value="")

display(avg_label)
display(counter)

interact(update_histogram,t=t_slider,x=x_slider, dt=dt_slider, dx=dx_slider, nsim = nsim_slider)

Label(value='')

Label(value='')

interactive(children=(FloatSlider(value=0.0, description='t:', max=10.0), FloatSlider(value=0.0, description='…

<function __main__.update_histogram(t, x, dt, dx, nsim)>

Now we use Z to estimate $u_t$


In [6]:

# def P(t,q): return np.exp(-q*t)*np.array([[q*t +1 , t], [-q**2 *t , (1-q*t) ]])
# def G(t,s,q): return np.exp(q*(s-t))*np.array([[(t-s)*q**2/2, (t-s)*q],[(1-q*t+q*s)*q**2/2 ,(1-q*t+q*s)*q ]])

def Pt1(t,q): return -q**2 * t *np.exp(-q*t) 
def Pt2(t,q): return (1-q*t)*np.exp(-q*t)

def Gt(t,s,q): return (1-q*t+q*s)*np.exp(-q*(s-t))
# def Gt1(t,s,q): return G(t,s,q)[1][0]
# def Gt2(t,s,q): return G(t,s,q)[1][1]

def Zt(t,x,dx,dt):
    t0 = t-dt
    q =np.sqrt(2)/dx
    s = dt*U()+t0
    xnew = x+dx if U()<1/2 else x-dx
    samp = 5/10
    if U()<samp: 
        sol = Pt1(dt,q)*u(t0,x)/samp
    else:
        sol=dt*Gt(t,s,q)*q**2*u(s,xnew)/(1-samp)
    
    rr_rate = q if 1/q <1 else 1
    rr_rate *= 2
    if 1/rr_rate>U():
        rr = rr_rate
        global count
        count +=1
        sol += rr*dt*Pt2(dt,q)*Zt(t0,x,dx,dt) 
        sol += rr*dt*Gt(t,s,q)*2*q*Zt(s,x,dx,dt)
    return sol

def update_histogram(t,x,dt, dx,nsim):
    nsim = 10**int(nsim)
    global count
    count = 0
    zz = np.array([Zt(t, x, dx,dt) for _ in range(nsim)])
    avg_label.value = f"Updated Average of zz: {np.average(zz):.2f}, {ut(t,x):.2f}"
    counter.value = f"Updated Average of count: {count/nsim} "
    plt.hist(zz, bins=50)
    plt.xlabel("Values of Z")
    plt.ylabel("Frequency")
    plt.title("Histogram of Z")
    plt.show()

dt_slider = widgets.FloatSlider(value=1, min=0.1, max=10, step=0.01, description='dt:')
dx_slider = widgets.FloatSlider(value=1, min=0.1, max=10, step=0.01, description='dx:')
t_slider = widgets.FloatSlider(value=0, min=0, max=10.0, step=0.1, description='t:')
x_slider = widgets.FloatSlider(value=0, min=-50, max=50, step=1, description='x:')
nsim_slider = widgets.FloatSlider(value=0, min=0, max=5, step=1, description='nsim:')

avg_label = widgets.Label(value="")
counter = widgets.Label(value="")

display(avg_label)
display(counter)

interact(update_histogram,t=t_slider,x=x_slider, dt=dt_slider, dx=dx_slider, nsim = nsim_slider)

Label(value='')

Label(value='')

interactive(children=(FloatSlider(value=0.0, description='t:', max=10.0), FloatSlider(value=0.0, description='…

<function __main__.update_histogram(t, x, dt, dx, nsim)>

This is with ut RRVE estimated $u_t$ probably wrong ...

In [7]:

from random import random as U
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

# def u(t,x): return x**2 + 2*t #needs to satisfy wave equation
# def ut(t,x): return  2 #needs to satisfy wave equation

# def u(t,x): return x*t**2 + x**3/3 #needs to satisfy wave equation
# def ut(t,x): return 2*x*t  #needs to satisfy wave equation

def u(t,x): return np.sin(x+t) #needs to satisfy wave equation
def ut(t,x): return np.cos(x+t)  #needs to satisfy wave equation

def Z0(t,x,dx):
    q =np.sqrt(2)/dx
    P = np.exp(-q*t)*np.matrix([[q*t +1 , t], [-q**2 *t , (1-q*t) ]])
    sol = P @ np.array( [u(0,x), ut(0,x) ])
    sol = np.transpose(sol)
    samp = 1-1/2
    if U()<samp:
        s = t*U()
        G = np.exp(q*(s-t))*np.matrix([[(t-s)*q**2/2, (t-s)*q],[(1-q*t+q*s)*q**2/2 ,(1-q*t+q*s)*q ]])
        xnew = x+dx if U()<1/2 else x-dx
        sol +=  2 *t*G @ Z0(s,xnew,dx)/samp 
    return np.array(sol)
# Function to update and display the histogram
def update_histogram(t,x,dt, dx,nsim):
    nsim = 10**int(nsim)
    zz = np.array([Z0(t, x, dx) for _ in range(nsim)])
    zz0,zz1 = zip(*list(zz))
    avg_label.value = f"Updated Average of zz: {np.average(zz0):.2f},{np.average(zz1):.2f}, {u(t,x):.2f}, {ut(t,x):.2f}"
    zz00 = [float(z) for z in zz0]
    plt.hist(zz00, bins=int(nsim/5+1))
    plt.xlabel("Values of Z")
    plt.ylabel("Frequency")
    plt.title("Histogram of Z")
    plt.show()

# Create sliders for dt and dx
dt_slider = widgets.FloatSlider(value=1, min=0.1, max=10, step=0.01, description='dt:')
dx_slider = widgets.FloatSlider(value=1, min=0.1, max=10, step=0.01, description='dx:')
t_slider = widgets.FloatSlider(value=0, min=0, max=10.0, step=0.1, description='t:')
x_slider = widgets.FloatSlider(value=0, min=-50, max=50, step=1, description='x:')
nsim_slider = widgets.FloatSlider(value=0, min=0, max=5, step=1, description='nsim:')

# Create a label to display the average of zz
avg_label = widgets.Label(value="")

# Create an interactive interface
display(avg_label)
interact(update_histogram,t=t_slider,x=x_slider, dt=dt_slider, dx=dx_slider, nsim = nsim_slider)

Label(value='')

interactive(children=(FloatSlider(value=0.0, description='t:', max=10.0), FloatSlider(value=0.0, description='…

<function __main__.update_histogram(t, x, dt, dx, nsim)>

This is with euler integration

In [113]:
from random import random as U
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

# def u(t,x): return x**2 + 2*t #needs to satisfy wave equation
# def ut(t,x): return  2 #needs to satisfy wave equation

# def u(t,x): return x*t**2 + x**3/3 #needs to satisfy wave equation
# def ut(t,x): return 2*x*t  #needs to satisfy wave equation

def u(t,x): return np.sin(x+t) #needs to satisfy wave equation
def ut(t,x): return np.cos(x+t)  #needs to satisfy wave equation
def P(t,q): return np.exp(-q*t)*np.matrix([[q*t +1 , t], [-q**2 *t , (1-q*t) ]])
def G(t,s,q): return np.exp(q*(s-t))*np.matrix([[(t-s)*q**2/2, (t-s)*q],[(1-q*t+q*s)*q**2/2 ,(1-q*t+q*s)*q ]])

def ZZ(t, x, dx, dt):
    q = np.sqrt(2) / dx
    t0 = t - dt
    if t0 < 0: return np.transpose(np.matrix([u(0, x), ut(0, x)]))
    if U() < 1/3:
        PP = P(dt, q)
        sol = 3 * PP @ ZZ(t0, x, dx, dt)
    else:
        GG = G(t, t0, q)
        xnew = x + dx if U() < 1/2 else x - dx
        sol = 3 * dt * GG @ ZZ(t0, xnew, dx, dt)
    return np.matrix(sol)


# Function to update and display the histogram
def update_histogram(t,x,dt, dx,nsim):
    nsim = 10**int(nsim)
    zz = np.array([ZZ(t, x, dx,dt) for _ in range(nsim)])
    zz0,zz1 = zip(*list(zz))
    avg_label.value = f"Updated Average of zz: {np.average(zz0):.2f},{np.average(zz1):.2f}, {u(t,x):.2f}, {ut(t,x):.2f}"
    zz00 = [float(z) for z in zz0]
    plt.hist(zz00, bins=int(nsim/5+1))
    plt.xlabel("Values of Z")
    plt.ylabel("Frequency")
    plt.title("Histogram of Z")
    plt.show()

# Create sliders for dt and dx
dt_slider = widgets.FloatSlider(value=1, min=0.1, max=10, step=0.01, description='dt:')
dx_slider = widgets.FloatSlider(value=1, min=0.1, max=10, step=0.01, description='dx:')
t_slider = widgets.FloatSlider(value=0, min=0, max=10.0, step=0.1, description='t:')
x_slider = widgets.FloatSlider(value=0, min=-50, max=50, step=1, description='x:')
nsim_slider = widgets.FloatSlider(value=0, min=0, max=5, step=1, description='nsim:')

# Create a label to display the average of zz
avg_label = widgets.Label(value="")

# Create an interactive interface
display(avg_label)
interact(update_histogram,t=t_slider,x=x_slider, dt=dt_slider, dx=dx_slider, nsim = nsim_slider)

Label(value='')

interactive(children=(FloatSlider(value=0.0, description='t:', max=10.0), FloatSlider(value=0.0, description='…

<function __main__.update_histogram(t, x, dt, dx, nsim)>